# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
from skimage import io
from skimage import img_as_uint
from glob import glob
import numpy as np
import tifffile
from denoiser import Denoiser
from lsm_utils import normalize_16bit_images, compute_norm_range
import yaml

# Load data 
- SHG samples
- load model config data from yaml file

In [4]:
def sandbox_compute_norm_range(fname, percentiles=(0, 100), sample_r=0.1):
    max_val = []
    min_val = []
    fail_names = []
    try:
        img = img_as_uint(io.imread(fname))
    except Exception as e:
        print(e)
        print(fname)
        fail_names.append(fname)
    max_val.append(np.percentile(img, percentiles[1]))
    min_val.append(np.percentile(img, percentiles[0]))
    max_val: float | np.ndarray = np.percentile(np.array(max_val), 98)
    min_val: float | np.ndarray = np.percentile(np.array(min_val), 2)
    
    return min_val, max_val, fail_names

# vmin, vmax, fail_names = sandbox_compute_norm_range('sample_data/1B_C1.tif', percentiles=(1, 99.5), sample_r=0.05)
vmin, vmax, fail_names = compute_norm_range('sample_data', ext='tif', percentiles=(1, 99.5), sample_r=1)

sample_data\*.tif


100%|██████████| 1/1 [00:00<00:00,  2.50it/s]


In [5]:
config = yaml.load(open("model_config.yaml", "r"), Loader=yaml.FullLoader)
config['dataset'] = 'sample_data'
config['norm-range'] = [int(vmin), int(vmax)]
config['threads'] = 0

# Create model instance
* Create instance of denoiser with new config data

In [6]:
# Set background screening to false to allow PB522-14-MAX-Fused.tif to be accepted
denoiser = Denoiser(config, screen_bg=False)

c:\Users\lociuser\mambaforge\envs\lsm-run-time-enhancement\Lib\site-packages\torch\hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to C:\Users\lociuser/.cache\torch\hub\master.zip


RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [9]:
# Denoiser only works on grayscale images
denoiser.denoise()

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.39 GiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 10.78 GiB is allocated by PyTorch, and 105.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF